# English - Italian Embeddings (3 versions)
`w266 Final Project: Crosslingual Word Embeddings`

Instead of traning on randomly substituted words, here we'll choose the translation that is closest to the context embedding vector.

# Notebook Setup

In [138]:
# general imports
from __future__ import print_function
import time
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

# tell matplotlib not to open a new window
%matplotlib inline

# autoreload modules
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


__Base Paths__

In [139]:
BASE = '/home/mmillervedam/Data'
PROJ = '/home/mmillervedam/ProjectRepo'
GTT_BASE = PROJ + '/BaselineModels/data/ground_truth_translations/'
#PROJ = '/Users/mona/OneDrive/repos/final_proj/W266-Fall-2017-Final-Project'

# directory to save pickled embeddings
SAVE_TO = BASE + '/embeddings'

__Globals__ - _the parameters below fully determine all 3 models in this NB_

In [140]:
# Data
LANG = ('en','it')
FULL_TEXT = "/home/miwamoto/en_it_shuf.txt"
VOCAB_INDEX = BASE + '/vocab/en_it_small.pkl'
PANLEX = BASE + '/panlex/en_it_dict.pkl'
GTT_PATH = GTT_BASE + "%s-%s-clean.csv" % (LANG[1], LANG[0])

# Model
EMBEDDING_SIZE = 200

# Training
nBATCHES = 50000 # <<< 1 epoch with our 1 million sentence corpus
BATCH_SIZE = 48
WINDOW_SIZE = 4
MAX_EPOCHS = 5 # fail safe
ALPHA = 0.5 # authors use a much smaller learning rate but train longer

# Load Data

In [102]:
from parsing import Corpus, BilingualVocabulary, batch_generator, get_common_words

In [64]:
# load corpus
raw_data = Corpus(FULL_TEXT)

In [65]:
# load panlex dictionary
with open(PANLEX,'rb') as f:
    translations = pickle.load(f)

In [66]:
# load vocabulary
vocab = BilingualVocabulary([], languages = LANG)
with open(VOCAB_INDEX,'rb') as f:
    vocab.load_from_index(pickle.load(f))

In [67]:
# confirmations
print('... loaded %s panlex translations'%(len(translations)))
print('... loaded %s word %s vocabulary'%(vocab.size,vocab.language))

... loaded 525091 panlex translations
... loaded 20003 word ('en', 'it') vocabulary


In [77]:
# Validation Words (for training printout)
TEST_WORDS = vocab.to_ids(['en_the','en_last', 'it_si', 'it_suo'])
print('... test word ids:', TEST_WORDS)

... test word ids: [3, 239, 10020, 10040]


In [144]:
# Ground Truth Translations
gtt_df = pd.read_csv(GTT_PATH, names = [LANG[1], LANG[0]], sep=' ', header=None)
print('... loaded %s ground truth translations.'%(len(GTT_DF)))

... loaded 103613 ground truth translations.


In [142]:
# Evaluation Words (for reporting recall)
eval_words = [w for w in get_common_words(vocab) if w.startswith(LANG[1])]
EVAL_IDS = vocab.to_ids(eval_words)
print('... loaded %s evaluation words.' % (len(EVAL_IDS)))

... loaded 8516 evaluation words.


# Method 1: Random Translations

### Initialize the model

In [39]:
from models import BiW2V_random

# create model
model_1 = BiW2V_random(bilingual_dict = translations,
                       vocab = vocab, H = EMBEDDING_SIZE)

# intialize TF graphs
model_1.BuildCoreGraph()
model_1.BuildTrainingGraph()
model_1.BuildValidationGraph()

... TF graph created for BiW2V model.
... TF graph created for BiW2V training.
... TF graph created for BiW2V validation.


### Training

In [40]:
# fresh data generator
DATA_GENERATOR = batch_generator(raw_data, vocab, BATCH_SIZE, WINDOW_SIZE, MAX_EPOCHS)

In [41]:
# train
start = time.time()
model_1.train(nBATCHES, DATA_GENERATOR, TEST_WORDS, learning_rate = ALPHA)
tot = (time.time() - start)
print('... {} batches trained in {} seconds'.format(nBATCHES, tot))

... Model Initialized
	 <tf.Variable 'Embedding_Layer/ContextEmbeddings:0' shape=(20003, 200) dtype=float32_ref>
	 <tf.Variable 'Hidden_Layer/WordEmbeddings:0' shape=(20003, 200) dtype=float32_ref>
	 <tf.Variable 'Hidden_Layer/b:0' shape=(20003,) dtype=float32_ref>
... Starting Training
... STEP 0 : Average Loss : 0.000264484786987
   [en_the] closest:  it_cristina, en_commented, it_sacerdotale, it_amiga, it_vantaggio, en_muhammad, it_gemello, it_esistente,
   [en_last] closest:  en_n, en_ridge, it_kg, it_cancellare, it_professionistico, en_quad, it_consentire, it_arti,
   [it_si] closest:  it_compiti, it_limitazione, en_sculptor, en_switching, en_solidarity, en_hereford, it_u, it_pesante,
   [it_suo] closest:  it_abitanti, it_presentarsi, en_convert, en_regulate, it_notazione, en_son-in-law, it_compresa, it_girone,
... STEP 50000 : Average Loss : 4.10927392767
... STEP 100000 : Average Loss : 3.85516811703
   [en_the] closest:  en_a, en_its, en_an, en_one, en_his, it_la, it_riunirsi, 

### Save the Embeddings.

In [43]:
# context 
filename = SAVE_TO + '/en_it_rand_500K_V_dec19.pkl'
with open(filename, 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(model_1.context_embeddings, f, pickle.HIGHEST_PROTOCOL)

# word
filename = SAVE_TO + '/en_it_rand_500K_U_dec19.pkl'
with open(filename, 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(model_1.word_embeddings, f, pickle.HIGHEST_PROTOCOL)

### Evaluation

In [121]:
# load saved embeddings
with open(SAVE_TO + '/en_it_rand_500K_V_dec19.pkl','rb') as f:
    C_embedding = pickle.load(f)

In [229]:
# sanity checks
print('... C shape:', C_embedding.shape)
print('... eval IDs should be > 10003:', EVAL_IDS[:5])
print('... number to eval:', len(EVAL_IDS))
print('... GTT src language:', gtt_df.columns[0])

... C shape: (20003, 200)
... eval IDs should be > 10003: [19065, 10679, 17158, 15613, 13376]
... number to eval: 8516
... GTT src language: it


__Bilingual Induction Task.__

In [232]:
from models import evaluateBLI

In [233]:
src_nbrs, tgt_nbrs = evaluateBLI(C_embedding, vocab, gtt_df, 
                                 EVAL_IDS, top_k = 10, verbose=True)

... Evaluating 8516 'it' Ground Truth Translations
... TF graph created for BiW2V model.
... TF graph created for BiW2V validation.
... finding neighbors.
... Done. Total successful translation rate: 0


(array([[19065, 15359, 14739, 12500, 11830],
        [10679, 14462, 14183, 10859, 13836],
        [17158, 10289, 10657, 14685, 16737],
        ..., 
        [13539, 19729, 10873, 18593, 15290],
        [15296, 10101, 19645, 17042, 16285],
        [19768, 10548, 10093, 10720, 14907]]),
 array([[  28, 3125, 6106, 6872, 6342],
        [8353, 8809,   10,  153, 6040],
        [7202, 4800, 9888, 4377, 7412],
        ..., 
        [  46, 1117, 9994,  108, 5437],
        [ 157, 8692, 6712,  593, 2468],
        [ 188,  237,  155,  765, 7579]]))

# Method 2: Most Common Target Translation

### Initialize the model

In [46]:
from models import BiW2V_mle

# create model
model_2 = BiW2V_mle(bilingual_dict = translations,
                       vocab = vocab, H = EMBEDDING_SIZE)

# intialize TF graphs
model_2.BuildCoreGraph()
model_2.BuildTrainingGraph()
model_2.BuildValidationGraph()

... TF graph created for BiW2V model.
... TF graph created for BiW2V training.
... TF graph created for BiW2V validation.


### Training

In [50]:
# fresh data generator
DATA_GENERATOR = batch_generator(raw_data, vocab, BATCH_SIZE, WINDOW_SIZE, MAX_EPOCHS)

In [51]:
# train
start = time.time()
model_2.train(nBATCHES, DATA_GENERATOR, TEST_WORDS, learning_rate = ALPHA)
tot = (time.time() - start)
print('... {} batches trained in {} seconds'.format(nBATCHES, tot))

... Model Initialized
	 <tf.Variable 'Embedding_Layer/ContextEmbeddings:0' shape=(20003, 200) dtype=float32_ref>
	 <tf.Variable 'Hidden_Layer/WordEmbeddings:0' shape=(20003, 200) dtype=float32_ref>
	 <tf.Variable 'Hidden_Layer/b:0' shape=(20003,) dtype=float32_ref>
... Starting Training
... STEP 0 : Average Loss : 0.00252045059204
   [en_the] closest:  en_bern, en_mini, it_9, en_trout, en_gather, en_rival, en_homes, it_politica,
   [en_last] closest:  it_led, en_idaho, it_funzioni, en_up, it_svevia, it_trascrizione, en_climbing, it_foto,
   [it_si] closest:  en_6th, en_batting, en_reporter, en_shortened, it_alpini, it_tecnologie, en_addresses, en_credits,
   [it_suo] closest:  en_overhead, en_rico, en_venice, it_provenza, it_raven, it_battezzato, it_fire, it_normativa,
... STEP 5000 : Average Loss : 4.71148335021
... STEP 10000 : Average Loss : 4.22817595466
   [en_the] closest:  en_a, en_to, en_and, en_bern, en_mini, en_in, en_trout, it_politica,
   [en_last] closest:  it_led, en_idah

### Save the Embeddings.

In [52]:
# context 
filename = SAVE_TO + '/en_it_mle_50K_V_dec19.pkl'
with open(filename, 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(model_2.context_embeddings, f, pickle.HIGHEST_PROTOCOL)

# word
filename = SAVE_TO + '/en_it_mle_50K_U_dec19.pkl'
with open(filename, 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(model_2.word_embeddings, f, pickle.HIGHEST_PROTOCOL)

# Method 3: Closest Translation

### Initialize the model

In [56]:
from models import BiW2V_nn

# create model
model_3 = BiW2V_nn(bilingual_dict = translations,
                   vocab = vocab, H = EMBEDDING_SIZE)

# intialize TF graphs
model_3.BuildCoreGraph()
model_3.BuildTrainingGraph()
model_3.BuildValidationGraph()

... TF graph created for BiW2V model.
... TF graph created for BiW2V training.
... TF graph created for BiW2V validation.


### Training

In [57]:
# fresh data generator
DATA_GENERATOR = batch_generator(raw_data, vocab, BATCH_SIZE, WINDOW_SIZE, MAX_EPOCHS)

In [58]:
# train
nBATCHES = 5000 # Takes too long w/ nn so we'll only do 5K
start = time.time()
model_3.train(nBATCHES, DATA_GENERATOR, TEST_WORDS, learning_rate = ALPHA)
tot = (time.time() - start)
print('... {} batches trained in {} seconds'.format(nBATCHES, tot))

... Model Initialized
	 <tf.Variable 'Embedding_Layer/ContextEmbeddings:0' shape=(20003, 200) dtype=float32_ref>
	 <tf.Variable 'Hidden_Layer/WordEmbeddings:0' shape=(20003, 200) dtype=float32_ref>
	 <tf.Variable 'Hidden_Layer/b:0' shape=(20003,) dtype=float32_ref>
... Starting Training
... STEP 0 : Average Loss : 0.02340858078
   [en_the] closest:  en_nutrition, it_ma, en_prolonged, it_falco, it_vercelli, en_artists, en_beliefs, it_idraulico,
   [en_last] closest:  en_earn, en_renowned, it_decadimento, it_boy, en_korea, en_basement, en_resulting, en_told,
   [it_si] closest:  en_qualities, it_generato, it_scuole, en_employer, en_situation, en_chorus, en_to, en_inadequate,
   [it_suo] closest:  en_sat, it_rimpiazzare, en_playing, it_triangolare, it_realmente, en_profile, it_tutta, it_stirpe,
... STEP 500 : Average Loss : 6.30888293695
... STEP 1000 : Average Loss : 5.52120716763
   [en_the] closest:  en_nutrition, en_artists, it_falco, it_ma, en_beliefs, it_saturno, it_anime, it_siccit

### Save the Embeddings.

In [59]:
# context 
filename = SAVE_TO + '/en_it_nn_5K_V_dec19.pkl'
with open(filename, 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(model_2.context_embeddings, f, pickle.HIGHEST_PROTOCOL)

# word
filename = SAVE_TO + '/en_it_nn_5K_U_dec19.pkl'
with open(filename, 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(model_2.word_embeddings, f, pickle.HIGHEST_PROTOCOL)